# Websrapping

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import xlsxwriter
import os
import time

In [2]:
def scoresfixtures(link,ids):
    '''
    Description: This Class picks all the games that had in one season and combinate all links to one especific list
    
    Inputs:
        - link: The link of the main page that have all season games desired.
        - ids: The ID of the championship table
        
    Outputs:
        - especific list that has all the links os all matches of the season
    
    
    '''
    req = requests.get(link)
    if req.status_code == 200:
        content = req.content

    soup = BeautifulSoup(content, 'html.parser')
    tb = soup.find(id=ids)

    s1= []
    s2= []
    for i in tb.find_all("a"):
            s1.append(str(i))
            s2.append(str(i.get_text('href')))


    # Calling DataFrame constructor after zipping 
    # both lists, with columns specified 
    di = pd.DataFrame(list(zip(s1, s2)), 
                   columns =['Codes', 'ID']) 

    s4=[]
    for i in di["Codes"]:
        i = i.replace('<a href="','')
        i = i.replace('</a>','')
        s4.append(str(i))


    s5 = []

    for i in di['Codes']:
        if "matches" in i:
            s5.append(str(i))
        else:
            s5.append(0)

    s6 = []
    for i in di["Codes"]:
        if '<a href="/en/squads/' in i:
            i = i.replace('<a href="/en/squads/','')
            i = i[0:8]
            s6.append(str(i))
        else:
            s6.append(0)        

    # Calling DataFrame constructor after zipping 
    # both lists, with columns specified 
    da = pd.DataFrame(list(zip(s1, s2,s4,s5,s6)), 
                   columns =['CODES', 'ID','URL_FINAL','PARTIDAS_2019',"TEAM_CODE"])        

    s9 = []
    for i in da["URL_FINAL"]:
        if 'Match Report' in i:
            s9.append(str(i))
        else:
            pass
    return s9


In [3]:
# test
scoresfixtures("https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures","div_sched_10728_1")

['/en/matches/bf52349b/Fulham-Arsenal-September-12-2020-Premier-League">Match Report',
 '/en/matches/db261cb0/Crystal-Palace-Southampton-September-12-2020-Premier-League">Match Report',
 '/en/matches/21b58926/Liverpool-Leeds-United-September-12-2020-Premier-League">Match Report',
 '/en/matches/78495ced/West-Ham-United-Newcastle-United-September-12-2020-Premier-League">Match Report',
 '/en/matches/7dd01ca9/West-Bromwich-Albion-Leicester-City-September-13-2020-Premier-League">Match Report',
 '/en/matches/fc7f9aa1/Tottenham-Hotspur-Everton-September-13-2020-Premier-League">Match Report',
 '/en/matches/a3eb7a37/Sheffield-United-Wolverhampton-Wanderers-September-14-2020-Premier-League">Match Report',
 '/en/matches/9d7641eb/Brighton-and-Hove-Albion-Chelsea-September-14-2020-Premier-League">Match Report',
 '/en/matches/45bd1880/Everton-West-Bromwich-Albion-September-19-2020-Premier-League">Match Report',
 '/en/matches/583c2b60/Leeds-United-Fulham-September-19-2020-Premier-League">Match Report

In [4]:
len(scoresfixtures("https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures", "div_sched_10728_1"))

337

### For testing

In [21]:
url_pg

'https://fbref.com/en/matches/f4fca137/Montpellier-Marseille-March-14-2020-Ligue-1">Match Report'

In [30]:
pg = 'https://fbref.com'
url = scoresfixtures("https://fbref.com/en/comps/13/3243/schedule/2019-2020-Ligue-1-Scores-and-Fixtures", "div_sched_3243_1")
url_pg = pg+ url
req = requests.get(url_pg)

content = req.content
soup = BeautifulSoup(content, 'html.parser')

In [31]:
percentage = str(soup.find_all('strong')).split('%')
possession_1 = int(percentage[0][-2:])
possession_2 = int(percentage[1][-2:])

ValueError: invalid literal for int() with base 10: '>]'

In [7]:
save = str(soup.find_all('td')).split("save_pct")
save1 = save[1].split('>')[1].split('<')[0]
save2 = save[2].split('>')[1].split('<')[0]
save_1 = float(save1) if save1 != '' else 0
save_2 = float(save2) if save2 != '' else 0

save_2

66.7

In [8]:
#save_1 = float(percentage[6].split('>')[-1])
#save_2 = float(percentage[7].split('>')[-1]) 

## Scrape datasets

In [3]:
def get_df(url, league, season):
    '''
    Description: This function goes to de URL of the match and treat all data in order to append it in one single Dataframe.
    
    Input:
        - urls: urls of all the match reports
        - league: league
        - season: season
    Output:
        - Dataframe treated from the match saved on my machine excel file
    
    '''
    df_all= pd.DataFrame()
    #ind = 0
    # make the request
    pg = 'https://fbref.com'
    url_pg = pg+ url
    req = requests.get(url_pg)
    if req.status_code == 200:
        content = req.content
    # accessing data from site
    soup = BeautifulSoup(content, 'html.parser')


    table_geral = soup.find_all(class_ = "table_wrapper tabbed")
    table_1 = table_geral[0]
    table_2 = table_geral[1]
    table_3 = soup.find(class_='venuetime')


    # possession
    percentage = str(soup.find_all('strong')).split('%')
    possession_1 = int(percentage[0][-2:])
    possession_2 = int(percentage[1][-2:])

    # save percentage
    save = str(soup.find_all('td')).split("save_pct")
    save1 = save[1].split('>')[1].split('<')[0]
    save2 = save[2].split('>')[1].split('<')[0]
    save_1 = float(save1) if save1 != '' else 0
    save_2 = float(save2) if save2 != '' else 0



    #collecting data
    date = table_3.get('data-venue-date')        


    #treating data
    match = str(soup.title)
    match = match.replace(" ","_")
    match = match.replace("<title>","")
    match = match.replace(".","")
    match_final = match.split("Report")[0]


    #treating data
    match_final = match_final.split("_Match")
    match_final = match_final[0]    


    # STR transform and reading tables
    table_str_1 = str(table_1)
    table_str_2 = str(table_2)
    df_1 = pd.read_html(table_str_1)[0]
    df_2 = pd.read_html(table_str_2)[0]

    #treating data
    team = str(match_final)
    team = team.replace("_"," ")
    team = team.split(" vs ")
    team_1 = str(team[0])
    team_2 = str(team[1])


    #Dtframe transforming
    df_1 = pd.DataFrame(df_1).tail(1)      #use only the summary row
    df_1.columns = df_1.columns.droplevel()
    df_1['League'] = league
    df_1['Season'] = season
    df_1['Team'] = str(team_1)
    df_1['Home or Away'] = 'Home'
    df_1['Match'] = str(match_final)
    df_1['Date'] = str(date)
    df_1['Possession'] = possession_1
    df_1['Save%'] = save_1


    df_2 = pd.DataFrame(df_2).tail(1)      #use only the summary row
    df_2.columns = df_2.columns.droplevel()
    df_2['League'] = league
    df_2['Season'] = season
    df_2['Team'] = str(team_2)
    df_2['Home or Away'] = 'Away'
    df_2['Match'] = str(match_final)
    df_2['Date'] = str(date)
    df_2['Possession'] = possession_2
    df_2['Save%'] = save_2


    df_all = df_all.append(df_1).append(df_2)
    
    #write to csv files
    if not os.path.exists(league + season + '.csv'):
        df_all.to_csv(league + season + '.csv',index=False)
    else:
        df_all.to_csv(league + season + '.csv',mode='a+', index=False, header=False)

sometimes the scraper might interrupt, probably due to the webpage is updating

In each section below, there are two cells scraping the same data, the first one is usual one, the second is using multi-processing.

#### 1. English Premier League(EPL) 2020-2021

In [32]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures", "div_sched_10728_1")
get_df(urls, league = 'EPL', season = '2020-2021')
    
print('Time for scraping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
Time for scraping: 594.9537224769592 seconds


In [9]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures", "div_sched_10728_1")
get_df_new = partial(get_df, league = 'EPL', season = '2020-2021')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

Time for scraping: 229.59193205833435 seconds


#### 2. EPL 2019-2020

In [14]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/9/3232/schedule/2019-2020-Premier-League-Scores-and-Fixtures", "div_sched_3232_1")
get_df(urls, league = 'EPL', season = '2019-2020')  
    
print('Time for scriping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scriping: 707.4274218082428 seconds


In [ ]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/9/3232/schedule/2019-2020-Premier-League-Scores-and-Fixtures", "div_sched_3232_1")
get_df_new = partial(get_df, league = 'EPL', season = '2019-2020')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

#### 3. EPL 2018-2019

In [16]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/9/1889/schedule/2018-2019-Premier-League-Scores-and-Fixtures", "div_sched_1889_1")
get_df(urls, league = 'EPL', season = '2018-2019')
    

print('Time for scriping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scriping: 507.9384958744049 seconds


In [ ]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/9/1889/schedule/2018-2019-Premier-League-Scores-and-Fixtures", "div_sched_1889_1")
get_df_new = partial(get_df, league = 'EPL', season = '2018-2019')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

#### 4. Spanish LaLiga 2020-2021

In [40]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/12/schedule/La-Liga-Scores-and-Fixtures", "div_sched_10731_1")[:335]
get_df(urls, league = 'LaLiga ', season = '2020-2021')
    

print('Time for scriping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
Time for scriping: 347.338219165802 seconds


In [ ]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/12/schedule/La-Liga-Scores-and-Fixtures", "div_sched_10731_1")[:335]
get_df_new = partial(get_df, league = 'LaLiga ', season = '2020-2021')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

#### 5. LaLiga 2019-2020

In [10]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/12/3239/schedule/2019-2020-La-Liga-Scores-and-Fixtures", "div_sched_3239_1")
get_df(urls, league = 'LaLiga ', season = '2019-2020')
    

print('Time for scriping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scriping: 634.6203856468201 seconds


In [ ]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/12/3239/schedule/2019-2020-La-Liga-Scores-and-Fixtures", "div_sched_3239_1")
get_df_new = partial(get_df, league = 'LaLiga ', season = '2019-2020')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

#### 6. LaLiga 2018-2019

In [11]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/12/1886/schedule/2018-2019-La-Liga-Scores-and-Fixtures", "div_sched_1886_1")
get_df(urls, league = 'LaLiga ', season = '2018-2019')
    

print('Time for scriping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scriping: 727.3283274173737 seconds


In [ ]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/12/1886/schedule/2018-2019-La-Liga-Scores-and-Fixtures", "div_sched_1886_1")
get_df_new = partial(get_df, league = 'LaLiga ', season = '2018-2019')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

#### 7. Italian Serie A 2020-2021

In [45]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/11/schedule/Serie-A-Scores-and-Fixtures", "div_sched_10730_1")
get_df(urls, league = 'Serie A ', season = '2020-2021')
    

print('Time for scriping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
Time for scriping: 697.5135340690613 seconds


In [ ]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/11/schedule/Serie-A-Scores-and-Fixtures", "div_sched_10730_1")
get_df_new = partial(get_df,league = 'Serie A ', season = '2020-2021')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

#### 8. Serie A 2019-2020

In [12]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/11/3260/schedule/2019-2020-Serie-A-Scores-and-Fixtures", "div_sched_3260_1")
get_df(urls, league = 'Serie A ', season = '2019-2020')
    

print('Time for scriping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scriping: 726.3007817268372 seconds


In [ ]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/11/3260/schedule/2019-2020-Serie-A-Scores-and-Fixtures", "div_sched_3260_1")
get_df_new = partial(get_df, league = 'Serie A ', season = '2019-2020')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

#### 9. Serie A 2018-2019

In [13]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/11/1896/schedule/2018-2019-Serie-A-Scores-and-Fixtures", "div_sched_1896_1")
get_df(urls, league = 'Serie A ', season = '2018-2019')
    

print('Time for scriping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scriping: 713.4366507530212 seconds


In [ ]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/11/1896/schedule/2018-2019-Serie-A-Scores-and-Fixtures", "div_sched_1896_1")
get_df_new = partial(get_df, league = 'Serie A ', season = '2018-2019')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

#### 10. Germany Bundesliga 2020-2021

In [55]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/20/schedule/Bundesliga-Scores-and-Fixtures", "div_sched_10737_1")
get_df(urls, league = 'Bundesliga ', season = '2020-2021')
    

print('Time for scriping: {} seconds'.format(time.time() - start))


scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
Time for scriping: 545.3423707485199 seconds


In [ ]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/20/schedule/Bundesliga-Scores-and-Fixtures", "div_sched_10737_1")
get_df_new = partial(get_df, league = 'Bundesliga ', season = '2020-2021')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

#### 11. Bundesliga 2019-2020

In [14]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/20/3248/schedule/2019-2020-Bundesliga-Scores-and-Fixtures", "div_sched_3248_1")
get_df(urls, league = 'Bundesliga ', season = '2019-2020')
    

print('Time for scriping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
Time for scriping: 586.717095375061 seconds


In [ ]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/20/3248/schedule/2019-2020-Bundesliga-Scores-and-Fixtures", "div_sched_3248_1")
get_df_new = partial(get_df, 'Bundesliga ', season = '2019-2020')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

#### 12. Bundesliga 2018-2019

In [15]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/20/2109/schedule/2018-2019-Bundesliga-Scores-and-Fixtures", "div_sched_2109_1")
get_df(urls, league = 'Bundesliga ', season = '2018-2019')
    

print('Time for scriping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
Time for scriping: 570.7660129070282 seconds


In [ ]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/20/2109/schedule/2018-2019-Bundesliga-Scores-and-Fixtures", "div_sched_2109_1")
get_df_new = partial(get_df, league = 'Bundesliga ', season = '2018-2019')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

#### 13. France Ligue 1 2020-2021

In [59]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/13/schedule/Ligue-1-Scores-and-Fixtures", "div_sched_10732_1")
get_df(urls, league = 'Ligue 1 ', season = '2020-2021')
    

print('Time for scriping: {} seconds'.format(time.time() - start))


scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scriping: 741.0207390785217 seconds


In [ ]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/13/schedule/Ligue-1-Scores-and-Fixtures", "div_sched_10732_1")
get_df_new = partial(get_df, league = 'Ligue 1 ', season = '2020-2021')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

#### 14. Ligue 1 2019-2020

In [38]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/13/3243/schedule/2019-2020-Ligue-1-Scores-and-Fixtures", "div_sched_3243_1")[:279]
get_df(urls, league = 'Ligue 1 ', season = '2019-2020')
    

print('Time for scriping: {} seconds'.format(time.time() - start))


Time for scriping: 8.902738571166992 seconds


In [ ]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/13/3243/schedule/2019-2020-Ligue-1-Scores-and-Fixtures", "div_sched_3243_1")[:279]
get_df_new = partial(get_df, league = 'Ligue 1 ', season = '2019-2020')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

#### 15. Ligue 1 2018-2019

In [17]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/13/2104/schedule/2018-2019-Ligue-1-Scores-and-Fixtures", "div_sched_2104_1")
get_df(urls, league = 'Ligue 1 ', season = '2018-2019')
    

print('Time for scriping: {} seconds'.format(time.time() - start))


scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scriping: 705.0203380584717 seconds


In [ ]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/13/2104/schedule/2018-2019-Ligue-1-Scores-and-Fixtures", "div_sched_2104_1")
get_df_new = partial(get_df, league = 'Ligue 1 ', season = '2018-2019')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

## Extra 2016-2018 data for supplement

#### 1. EPL 2017-2018

In [ ]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/9/1631/schedule/2017-2018-Premier-League-Scores-and-Fixtures", "div_sched_1631_1")
get_df(urls, league = 'EPL', season = '2017-2018')
    
print('Time for scraping: {} seconds'.format(time.time() - start))

In [6]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/9/1631/schedule/2017-2018-Premier-League-Scores-and-Fixtures", "div_sched_1631_1")
get_df_new = partial(get_df, league = 'EPL', season = '2017-2018')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

Time for scraping: 242.00849509239197 seconds


####  2. LaLiga 2017-2018

In [19]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/12/1652/schedule/2017-2018-La-Liga-Scores-and-Fixtures", "div_sched_1652_1")
get_df(urls, league = 'LaLiga ', season = '2017-2018')
    
print('Time for scraping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scraping: 710.4342136383057 seconds


In [7]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/12/1652/schedule/2017-2018-La-Liga-Scores-and-Fixtures", "div_sched_1652_1")
get_df_new = partial(get_df, league = 'LaLiga ', season = '2017-2018')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

Time for scraping: 275.07933497428894 seconds


####  3. Serie A 2017-2018

In [20]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/11/1640/schedule/2017-2018-Serie-A-Scores-and-Fixtures", "div_sched_1640_1")
get_df(urls, league = 'Serie A ', season = '2017-2018')
    

print('Time for scriping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scriping: 713.9229116439819 seconds


In [8]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/11/1640/schedule/2017-2018-Serie-A-Scores-and-Fixtures", "div_sched_1640_1")
get_df_new = partial(get_df, league = 'Serie A ', season = '2017-2018')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

Time for scraping: 330.8611168861389 seconds


####  4. Bundesliga 2017-2018

In [21]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/20/1634/schedule/2017-2018-Bundesliga-Scores-and-Fixtures", "div_sched_1634_1")
get_df(urls, league = 'Bundesliga ', season = '2017-2018')
    

print('Time for scriping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
Time for scriping: 570.6511466503143 seconds


In [21]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/20/1634/schedule/2017-2018-Bundesliga-Scores-and-Fixtures", "div_sched_1634_1")
get_df_new = partial(get_df, league = 'Bundesliga ', season = '2017-2018')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
Time for scriping: 570.6511466503143 seconds


####  5. Ligue 1 2017-2018

In [22]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/13/1632/schedule/2017-2018-Ligue-1-Scores-and-Fixtures", "div_sched_1632_1")
get_df(urls, league = 'Ligue 1 ', season = '2017-2018')
    

print('Time for scriping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scriping: 708.0039551258087 seconds


In [22]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/13/1632/schedule/2017-2018-Ligue-1-Scores-and-Fixtures", "div_sched_1632_1")
get_df_new = partial(get_df, league = 'Ligue 1 ', season = '2017-2018')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scriping: 708.0039551258087 seconds


####  6. EPL 2016-2017

In [23]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/9/1526/schedule/2016-2017-Premier-League-Scores-and-Fixtures", "div_sched_1526_1")
get_df(urls, league = 'EPL', season = '2016-2017')
    
print('Time for scraping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scraping: 312.6351389884949 seconds


In [23]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/9/1526/schedule/2016-2017-Premier-League-Scores-and-Fixtures", "div_sched_1526_1")
get_df_new = partial(get_df, league = 'EPL', season = '2016-2017')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scraping: 312.6351389884949 seconds


####  7. LaLiga 2016-2017

In [24]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/12/1547/schedule/2016-2017-La-Liga-Scores-and-Fixtures", "div_sched_1547_1")
get_df(urls, league = 'LaLiga ', season = '2016-2017')
    
print('Time for scraping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scraping: 311.80960607528687 seconds


In [24]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/12/1547/schedule/2016-2017-La-Liga-Scores-and-Fixtures", "div_sched_1547_1")
get_df_new = partial(get_df, league = 'LaLiga ', season = '2016-2017')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scraping: 311.80960607528687 seconds


####  8. Serie A 2016-2017

In [25]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/11/1535/schedule/2016-2017-Serie-A-Scores-and-Fixtures", "div_sched_1535_1")
get_df(urls, league = 'Serie A ', season = '2016-2017')
    

print('Time for scriping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scriping: 316.1644470691681 seconds


In [25]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/11/1535/schedule/2016-2017-Serie-A-Scores-and-Fixtures", "div_sched_1535_1")
get_df_new = partial(get_df, league = 'Serie A ', season = '2016-2017')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scriping: 316.1644470691681 seconds


####  9. Bundesliga 2016-2017

In [26]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/20/1529/schedule/2016-2017-Bundesliga-Scores-and-Fixtures", "div_sched_1529_1")
get_df(urls, league = 'Bundesliga ', season = '2016-2017')
    

print('Time for scriping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
Time for scriping: 259.5699129104614 seconds


In [26]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/20/1529/schedule/2016-2017-Bundesliga-Scores-and-Fixtures", "div_sched_1529_1")
get_df_new = partial(get_df, league = 'Bundesliga ', season = '2016-2017')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
Time for scriping: 259.5699129104614 seconds


####  10. Ligue 1 2016-2017

In [27]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/13/1527/schedule/2016-2017-Ligue-1-Scores-and-Fixtures", "div_sched_1527_1")
get_df(urls, league = 'Ligue 1 ', season = '2016-2017')
    

print('Time for scriping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scriping: 313.567583322525 seconds


In [27]:
start = time.time()

urls = scoresfixtures("https://fbref.com/en/comps/13/1527/schedule/2016-2017-Ligue-1-Scores-and-Fixtures", "div_sched_1527_1")
get_df_new = partial(get_df, league = 'Ligue 1 ', season = '2016-2017')
pool=Pool(8)
pool.map(get_df_new, urls)
pool.close()
pool.join()

print('Time for scraping: {} seconds'.format(time.time() - start))

scraped 50 matches
scraped 100 matches
scraped 150 matches
scraped 200 matches
scraped 250 matches
scraped 300 matches
scraped 350 matches
Time for scriping: 313.567583322525 seconds
